처음에 시도해본 CNN 모델입니다. 

In [1]:
import pandas as pd
import numpy as np
import pywt
import matplotlib.pyplot as plt
import os

# 1.Data Load

In [7]:
#!wget -r -N -c -np https://physionet.org/files/ludb/1.0.0/

# !pip install wfdb

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
from pathlib import Path
import re
import wfdb
from wfdb import processing

data_dir = Path('C:/Users/User/Desktop/3-2/딥러닝/final_project/dataset')

num_records = 200
records = []

In [8]:
from tqdm.notebook import tqdm
for i in tqdm(range(1,num_records+1)):
  record = wfdb.io.rdrecord(f'{data_dir}/{i}')
  age = 99 if record.comments[0][7:] == '>89' else int(record.comments[0][7:])
  data = {'id': i,
          'age(10)' : age //10 * 10, 
          'sex':record.comments[1][-1],
          'dignosis':record.comments[3:]}
  cols = ['i', 'ii',  'iii',  'avr',  'avl',  'avf',  'v1',  'v2',  'v3',  'v4',  'v5',  'v6']
  for col, j in zip(cols, range(12)):
      ann = wfdb.rdann(f'{data_dir}/{i}', f'atr_{col}')
      data[col] = {'signal':record.p_signal[:, j], 'anno': ann.__dict__['symbol'], 'anno_idx': ann.__dict__['sample']}
  records.append(data)

In [9]:
df=pd.DataFrame(records)

# 2.Preprocessing

DWT(Discrete Wavelet Transform) 이용해서 신호의 노이즈 제거.


In [10]:
from skimage.restoration import denoise_wavelet

In [11]:
denoised_data = []


for i in range(len(df)):

  data = {
      'id' : i+1
  }

  for colname in cols:

    data[colname] = denoise_wavelet(df[colname][i]['signal'], method='BayesShrink',
                                mode='soft', wavelet_levels=8, wavelet='db4',
                                rescale_sigma='True')
  
  denoised_data.append(data)


denoised_signals = pd.DataFrame(denoised_data)


denoised_signals.head()

,id,i,ii,iii,avr,avl,avf,v1,v2,v3,v4,v5,v6
0,1,"[-119.90064831203651, -42.98888452235324, 34.8...","[24.96133275294272, 97.11975027867946, 194.915...","[144.96063320938558, 140.1212408015322, 159.92...","[47.02106507781347, -27.09104484724255, -114.9...","[-131.8993985424003, -90.97190632975322, -62.0...","[84.96147711250939, 118.14174949753709, 177.91...","[149.95359244940772, 187.1541340593624, 236.88...","[61.91886341921407, 165.24208154940555, 316.83...","[64.92926957109226, 210.20038117298083, 401.84...","[144.92639059053806, 310.152629242335, 501.905...","[104.92360277790587, 234.9927326257242, 387.08...","[-24.8363932770309, 66.9076089355276, 152.1150..."
1,2,"[-162.0311090407124, -140.14318580821413, -191...","[55.048161274534976, 51.91047674834644, 35.089...","[216.91595616176974, 192.1811979662824, 226.96...","[52.99460343390038, 44.066223639392206, 77.939...","[-189.0426954690825, -166.15580845689013, -208...","[135.89703863398407, 122.19038567888818, 130.9...","[92.1409732400953, 106.9244341112607, 107.0672...","[65.13733417147752, 86.9062744830752, 82.08252...","[-31.907423555531963, -12.11194634724055, -21....","[-11.854303046496149, 6.922483743794106, 0.090...","[142.17369392741097, 159.90409312746883, 155.1...","[-1.799893062787706, 6.896407172484739, 5.1287..."
2,3,"[-13.196292165997493, -26.884378386710367, -20...","[-69.91195543703883, -42.158511588886455, -74....","[-56.90877215879811, -15.163640059095933, -54....","[40.90324743516223, 34.20982193029279, 46.9717...","[21.877096098219436, -5.8204928778199285, 16.9...","[-63.89826266348706, -29.16194682962693, -64.9...","[-21.82964297467621, 9.890043885373624, -9.908...","[45.15278751061051, 59.814061701447926, 52.100...","[-89.86717584889905, -80.18277335735694, -81.8...","[-19.890658040670434, -1.9756233818768996, -0....","[-20.101182802430017, -9.851206183100071, 4.87...","[-65.05908929693008, -56.85798850738285, -35.1..."
3,4,"[-99.90481239147844, -48.22663908931325, -92.9...","[340.0812629487586, 406.78758973614515, 377.01...","[439.9667522043796, 455.1857846060954, 469.864...","[-120.08713580730729, -178.79949260538027, -14...","[-269.91059160118294, -251.1750079220973, -280...","[390.0934877171292, 430.80601484750383, 423.03...","[-7.020309532580591, -9.883700537005257, 9.892...","[-357.0307740631268, -356.79074189312445, -340...","[-420.0470585848426, -424.80503514209676, -415...","[-395.07150680443647, -401.8562313429786, -387...","[-165.07652876571626, -174.84992950111325, -16...","[-285.0609536032772, -289.8951522753783, -280...."
4,5,"[92.12628446383064, 129.83362521497065, 105.08...","[52.142547369437345, 64.95561774641003, 60.094...","[-40.13537219404688, -64.74507315932857, -45.0...","[-72.09126231359882, -96.87587035936343, -82.0...","[66.1070470612472, 97.86191999581112, 75.08299...","[5.884695871738978, 0.2459985115122736, 7.9251...","[-17.13795519768861, -31.773409880312084, -20....","[326.88986589219513, 310.2092606028747, 324.92...","[511.9341014257459, 507.1729479309501, 521.888...","[496.8744889541595, 485.2312128860184, 506.885...","[266.89243932606524, 247.22324977196504, 266.9...","[104.85612497721111, 87.25622647077475, 106.86..."


### LVH 기준으로 labeling

In [12]:
df.head()

,id,age(10),sex,dignosis,i,ii,iii,avr,avl,avf,v1,v2,v3,v4,v5,v6
0,1,50,F,[Electric axis of the heart: left axis deviati...,"{'signal': [-120.0052369730296, -42.9955485729...","{'signal': [24.9907986750092, 97.0003680529996...","{'signal': [145.00309434953022, 139.9958742006...","{'signal': [47.00970671119925, -26.99091952823...","{'signal': [-131.99039766203944, -90.992589500...","{'signal': [85.0052668969403, 118.001923562360...","{'signal': [150.00429181853772, 186.9976740437...","{'signal': [62.007805547359816, 165.0103266384...","{'signal': [65.01161660381521, 209.99510516990...","{'signal': [144.99363597793806, 309.9985857728...","{'signal': [105.00697831184983, 235.0156181265...","{'signal': [-24.989606191375028, 67.0094956056..."
1,2,60,M,"[Electric axis of the heart: normal., Left atr...","{'signal': [-161.99997231361294, -140.00083059...","{'signal': [55.00456740180086, 52.003131932663...","{'signal': [216.99565555580728, 191.9991887206...","{'signal': [53.00530428878182, 43.997785077334...","{'signal': [-188.99679603056862, -165.99881762...","{'signal': [135.9938108439172, 122.00373928180...","{'signal': [92.00591873126685, 106.99244982357...","{'signal': [64.99081335982183, 86.993127646602...","{'signal': [-32.00735218386798, -12.0027570689...","{'signal': [-11.990844373968661, 7.00399788199...","{'signal': [141.9956753532416, 160.00726988032...","{'signal': [-2.0072642199961583, 7.00812076809..."
2,3,50,M,"[Electric axis of the heart: vertical., Left a...","{'signal': [-12.995302667933347, -26.999059158...","{'signal': [-70.00630374812768, -41.9864324833...","{'signal': [-57.004995283091816, -15.001314548...","{'signal': [41.00108545870253, 33.995642768612...","{'signal': [22.002115882921146, -5.99883473114...","{'signal': [-63.98523919563108, -29.0113880080...","{'signal': [-22.000055696343537, 9.99749366454...","{'signal': [45.00702125309645, 59.996213256756...","{'signal': [-90.01391604572919, -80.0172629428...","{'signal': [-19.997401467605904, -1.9771442128...","{'signal': [-20.007315288953915, -9.9853694220...","{'signal': [-64.99265926612877, -57.0075580499..."
3,4,50,M,[Electric axis of the heart: left axis deviati...,"{'signal': [-100.00813074233677, -48.008337706...","{'signal': [339.9995610524115, 406.99682456073...","{'signal': [439.99107718834966, 455.0031048972...","{'signal': [-119.99942625184676, -178.99508010...","{'signal': [-269.9960617369615, -250.998630984...","{'signal': [389.99483352326524, 431.0052876046...","{'signal': [-6.9953018040586255, -9.9932882915...","{'signal': [-356.98399763718726, -356.98399763...","{'signal': [-419.98453442975426, -425.00141453...","{'signal': [-394.9824477850411, -402.011015263...","{'signal': [-164.9860246768689, -174.992664885...","{'signal': [-284.9987530135506, -290.007481918..."
4,5,60,M,"[Electric axis of the heart: horizontal., Left...","{'signal': [92.00258910561116, 129.99832706640...","{'signal': [51.99879512887818, 65.002797022247...","{'signal': [-40.00193768119182, -64.9992671590...","{'signal': [-72.00004699851536, -97.0015786465...","{'signal': [66.00110254251149, 97.999067079413...","{'signal': [6.000550395312122, 0.0, 8.00245815...","{'signal': [-17.002450383617138, -31.994860929...","{'signal': [326.9989609547205, 310.00186187304...","{'signal': [511.9998504837086, 507.00039948884...","{'signal': [496.994767359638, 485.009192476311...","{'signal': [267.0138918247077, 247.00335106098...","{'signal': [105.01046863622516, 86.99015364803..."


In [13]:
LVH = [] # if data has LVH 1 , else 0
sex = [] # if data is M 1, else 0
for i in range(len(df)):
  
  if 'Left ventricular hypertrophy.' in df['dignosis'][i]:
    LVH.append(np.asarray([1,0],dtype=np.int32))

  else:
    LVH.append(np.asarray([0,1],dtype=np.int32))

  if df['sex'][i] == 'M':
    sex.append(1)

  else:
    sex.append(0)
    

denoised_signals['LVH'] = LVH
denoised_signals['sex'] = sex

In [14]:
denoised_signals['LVH'][1]

array([1, 0])

### 노이즈 제거한 신호에 CWT 적용

In [15]:
cwt = []

for i in range(len(df)):
  data = {'id' : i+1,
          'LVH' : denoised_signals['LVH'][i]}

  for colname in cols:
    data[colname] = pywt.cwt(denoised_signals[colname][i],np.arange(1,33),wavelet='morl')[0]
  cwt.append(data)

cwt_coef_32 = pd.DataFrame(cwt)
cwt_coef_32.head()

,id,LVH,i,ii,iii,avr,avl,avf,v1,v2,v3,v4,v5,v6
0,1,"[1, 0]","[[9.686967235895121, 24.444280990235875, -0.62...","[[9.159390923309633, 9.226241621630006, 1.7745...","[[-0.5389022107438208, -15.244689187658626, 2....","[[-9.371184262402418, -16.78004459162034, -0.6...","[[5.119406723108252, 19.756825203028328, -1.59...","[[4.256581107609934, -2.891735792457263, 2.095...","[[4.783707813286291, -12.167871864104498, -3.3...","[[13.075231433465675, 11.778372313782864, 8.88...","[[18.310668694927312, 16.492440543325117, 10.7...","[[20.76137210353792, 6.631659127671824, 8.3784...","[[16.335088318519006, 6.449260399370541, 7.296...","[[11.455410397431137, 13.907820438654275, 3.34..."
1,2,"[1, 0]","[[2.4307984868753643, 13.66622029538016, 0.077...","[[-0.4893980314241304, -8.764898378359488, 1.9...","[[-2.8856117787263003, -22.43572533014176, 1.8...","[[-0.9177632465825081, -2.4440618044060987, -1...","[[2.60904537882562, 18.055510046888774, -0.880...","[[-1.6595420621222523, -15.620132890780628, 1....","[[1.8122698099012648, -11.279641823649737, -1....","[[2.649963499378819, -8.571859885213042, -2.44...","[[2.3845204294572877, 2.76628989438431, -1.767...","[[2.2691343753075706, 0.673819651223466, -1.49...","[[2.1525807012616625, -18.052235266231385, -2....","[[1.0509484654239287, 0.05480662321308072, -0...."
2,3,"[1, 0]","[[-1.632608766797835, 2.4148229074086807, 0.25...","[[3.272801483799162, 4.554771632878164, -3.324...","[[4.941560358304136, 2.133353618001048, -3.604...","[[-0.7794838456002822, -3.4379008833069222, 1....","[[-3.286989849493533, 0.0730477681656111, 1.92...","[[4.107871556021779, 3.466416736429417, -3.459...","[[3.79862917884803, 0.30621071719923076, -2.64...","[[1.7648558198642474, -6.49582889994694, -1.64...","[[1.185460911760602, 10.856555938817943, -0.79...","[[2.2356124900994967, 2.6174265560991685, -3.5...","[[1.3550795674963196, 4.2012472841686055, -2.9...","[[1.1346498459999372, 10.60086074836341, -2.48..."
3,4,"[1, 0]","[[6.242210649457608, 6.525288125540515, -11.28...","[[8.149873173552749, -45.748427159077416, -14....","[[1.9297796120785904, -52.30822540029342, -3.3...","[[-7.140086894939728, 19.55351462636929, 12.94...","[[2.2182113950131495, 29.40678762797768, -4.04...","[[5.007081089580787, -49.0609644165384, -8.867...","[[-0.2322422599531475, 3.2235857623544297, -1....","[[0.11947696306672384, 45.988704727287875, 0.7...","[[-0.527629670193555, 52.87898451055677, 1.642...","[[-0.7488318598946293, 50.416056816059424, 1.3...","[[-1.117764934969042, 21.828074099123576, 0.10...","[[-0.5315205943739585, 36.264162799771164, 0.6..."
4,5,"[0, 1]","[[4.566061662325904, -14.521924370685403, -7.4...","[[1.5465062042982787, -6.998383340382717, -1.2...","[[-2.9859464092397534, 7.5164114829682855, 6.1...","[[-2.996583629924226, 10.752278074162476, 4.19...","[[3.84597631725147, -11.084219399874527, -6.82...","[[-0.686697963929173, 0.30802393278383094, 2.4...","[[-1.7540680195126273, 3.5810576132218603, 2.2...","[[-2.0104633714483384, -38.36683405546124, 2.0...","[[-0.5402334249066882, -61.16123395358437, -0....","[[-1.3655900643532703, -58.43605550878446, 0.9...","[[-2.37075354816168, -30.337202587164523, 3.62...","[[-2.102830082349721, -10.429677600182163, 3.2..."


# 3. Model - CNN based

In [16]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, SimpleRNN, concatenate, TimeDistributed, InputLayer, LSTM, Embedding, Conv3D, MaxPool3D, ConvLSTM2D, Flatten, BatchNormalization, Dropout, MaxPool1D, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
import numpy as np

In [33]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, MaxPooling1D

cnn_inputs = Input(shape=(32,5000,2))

cnn = Conv2D(12, (10,50), strides=[5,5], 
                 activation='relu', input_shape=(30,5000,2), data_format='channels_last')(cnn_inputs)

cnn = MaxPooling2D(pool_size=(1,5), data_format='channels_last')(cnn)
cnn = Dropout(0.3)(cnn)
cnn = Conv2D(12, (1,10), strides=[1,1], activation='relu', input_shape=(30,5000,2), data_format='channels_last')(cnn)
cnn = Dropout(0.2)(cnn)
cnn = MaxPooling2D(pool_size=(1,5), data_format='channels_last')(cnn)

cnn_outputs = Flatten()(cnn)

sex_age_inputs = Input(shape=(2))
sex_age_outputs = Dense(8, activation='sigmoid')(sex_age_inputs)

concatenated = tf.keras.layers.concatenate([cnn_outputs, sex_age_outputs])
concatenated = Dense(64, activation='sigmoid')(concatenated)
concatenated = Dense(16, activation='sigmoid')(concatenated)
outputs = Dense(2, activation='softmax')(concatenated)

cnn_model = Model(inputs=[cnn_inputs, sex_age_inputs], outputs=outputs)
cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', mode='min', patience=5)

cnn_model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 32, 5000, 2) 0                                            
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 5, 991, 12)   12012       input_3[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_2 (MaxPooling2D)  (None, 5, 198, 12)   0           conv2d_2[0][0]                   
__________________________________________________________________________________________________
dropout_2 (Dropout)             (None, 5, 198, 12)   0           max_pooling2d_2[0][0]            
_______________________________________________________________________________________

In [18]:
cols = ['i','iii']



signal_data = np.ndarray(shape=(200, 32, 5000, 2))
sex_age_data = []
x_data= []
y_data = []

for ii in range(200):
   
  for jj in range(2):
        
    signal_data[ii, :, :, jj] = cwt_coef_32[cols[jj]][ii]

  sex_age = []

  sex_age.append(denoised_signals['sex'][ii])
  sex_age.append(df['age(10)'][ii])
  sex_age = np.asarray(sex_age)

  
  
  x_data.append((signal_data[ii], sex_age))
  y_data.append(denoised_signals['LVH'][ii])     

In [28]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_data,y_data,test_size=0.2,shuffle=True,random_state=22)
x_train, x_valid, y_train, y_valid = train_test_split(x_train,y_train,test_size=0.2,shuffle=True,random_state=22)

In [29]:
signal_train = []
sex_age_train = []

signal_valid = []
sex_age_valid = []

signal_test = []
sex_age_test = []

for i in range(len(x_train)):
  signal_train.append(x_train[i][0])
  sex_age_train.append(x_train[i][1])

    
for k in range(len(x_valid)):
  signal_valid.append(x_test[k][0])
  sex_age_valid.append(x_test[k][1])
    
for k in range(len(x_test)):
  signal_test.append(x_test[k][0])
  sex_age_test.append(x_test[k][1])

signal_train = np.reshape(signal_train, (len(signal_train), 32, 5000, 2))
signal_valid = np.reshape(signal_valid, (len(signal_valid), 32, 5000, 2)) 
signal_test = np.reshape(signal_test, (len(signal_test),32, 5000, 2))

sex_age_train = np.reshape(sex_age_train, (len(sex_age_train), 2))
sex_age_valid = np.reshape(sex_age_valid, (len(sex_age_valid), 2))
sex_age_test = np.reshape(sex_age_test, (len(sex_age_test), 2))

y_train = np.reshape(y_train, (len(y_train), 2))
y_valid = np.reshape(y_valid, (len(y_valid), 2))
y_test = np.reshape(y_test, (len(y_test), 2))

In [34]:
cnn_model.fit([signal_train, sex_age_train], y_train, epochs=30, validation_data=([signal_valid, sex_age_valid],y_valid), verbose=2, callbacks=[callback])

Epoch 1/30
4/4 - 2s - loss: 0.6996 - accuracy: 0.5625 - val_loss: 0.6847 - val_accuracy: 0.6562
Epoch 2/30
4/4 - 2s - loss: 0.6929 - accuracy: 0.5078 - val_loss: 0.6889 - val_accuracy: 0.5625
Epoch 3/30
4/4 - 2s - loss: 0.6848 - accuracy: 0.5469 - val_loss: 0.6853 - val_accuracy: 0.5312
Epoch 4/30
4/4 - 2s - loss: 0.6832 - accuracy: 0.5469 - val_loss: 0.6904 - val_accuracy: 0.5312
Epoch 5/30
4/4 - 2s - loss: 0.6904 - accuracy: 0.5469 - val_loss: 0.6904 - val_accuracy: 0.5312
Epoch 6/30
4/4 - 1s - loss: 0.6886 - accuracy: 0.5469 - val_loss: 0.6924 - val_accuracy: 0.5312
Epoch 7/30
4/4 - 2s - loss: 0.6872 - accuracy: 0.5469 - val_loss: 0.6911 - val_accuracy: 0.5312
Epoch 8/30
4/4 - 2s - loss: 0.6878 - accuracy: 0.5469 - val_loss: 0.6903 - val_accuracy: 0.5312
Epoch 9/30
4/4 - 2s - loss: 0.6879 - accuracy: 0.5469 - val_loss: 0.6902 - val_accuracy: 0.5312


In [35]:
cnn_model.evaluate([signal_test, sex_age_test], y_test, verbose=2)

2/2 - 0s - loss: 0.6867 - accuracy: 0.5250


[0.686656653881073, 0.5249999761581421]